In [1]:
#Exercise 1
from base64 import b64encode, b64decode
input = "sdifusdf9a8sdfsadfas"
print("Input: %s" % input)
binput = input.encode()
encoded = b64encode(binput).decode()
print("Encoded: %s" % encoded)
decoded = b64decode(encoded).decode()
print("Decoded: %s" % decoded)

Input: sdifusdf9a8sdfsadfas
Encoded: c2RpZnVzZGY5YThzZGZzYWRmYXM=
Decoded: sdifusdf9a8sdfsadfas


In [2]:
#Exercise 2
#version 1
def XOR1(str1, str2):
    str1 = bytes.fromhex(str1)
    str2 = bytes.fromhex(str2)
    out = []
    for i in range(len(str1)):
        out.append(str1[i] ^ str2[i])
    return bytes.hex(bytes(out))
print(XOR1("1c0111001f010100061a024b53535009181c","686974207468652062756c6c277320657965"))

#version 2
def XOR2(str1, str2):
    xor_sequence = [a ^ b for a, b in zip(bytes.fromhex(str1), bytes.fromhex(str2))]
    return bytes.hex(bytes(xor_sequence))
print(XOR2("1c0111001f010100061a024b53535009181c","686974207468652062756c6c277320657965"))

#version 3
def XOR3(str1, str2):
    return ''.join(f"{x ^ y:02x}" for x, y in zip(bytes.fromhex(str1), bytes.fromhex(str2)))
print(XOR3("1c0111001f010100061a024b53535009181c","686974207468652062756c6c277320657965"))

746865206b696420646f6e277420706c6179
746865206b696420646f6e277420706c6179
746865206b696420646f6e277420706c6179


In [3]:
#Exercise 3
from collections import Counter
import string

def decrypt(string, key):
    return ''.join(chr(x ^ key) for x in bytes.fromhex(string))

def crackXOR(ct):
    scores = {}
    for c in range(0x00, 0xFF+1):
        decrypted = decrypt(ct, c)
        
        #Version 1
        #frequencies = {}
        #for byte in decrypted:
        #    if byte in byte_frequencies:
        #        frequencies[byte] += 1
        #    else:
        #        frequencies[byte] = 1
        #frequencies = dict(sorted(frequencies.items(), key = lambda x: x[1], reverse = True))
        
        # Version 2
        frequencies = Counter(decrypted)
        # count the frequencies of all letters, since XORing with the wrong key will end up
        # in many non-letter characters
        scores[c] = sum([frequencies[x] for x in string.ascii_uppercase + string.ascii_lowercase])
    
    best_key = max(scores, key=scores.get)
    return (best_key, decrypt(ct, best_key))

ct = "1b37373331363f782a373b332b783931367f2c783f312e3d783f37373c783a2a372c3076"
(key, decrypted_message) = crackXOR(ct)
print("Best key: %d (%c), decrypted text: %s" % (key, chr(key), decrypt(ct, key)))

Best key: 88 (X), decrypted text: Cooking rocks ain't give good broth.


In [4]:
# Exercise 4
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
BLOCKLEN = 16

# pad data using PKCS#7 to the nearest multiple of the block size
def pad(data, block_size):
    padder = padding.PKCS7(block_size * 8).padder()
    padded_data = padder.update(data) + padder.finalize()
    return padded_data

# unpad data
def unpad(padded_data, block_size):
    unpadder = padding.PKCS7(block_size * 8).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()
    return data

# pad and encrypt plaintext with AES in CBC mode with given key and IV
def AESencrypt(plaintext, key, iv):
    # pick an instance of AES in CBC mode
    aes = Cipher(algorithms.AES(k), modes.CBC(iv)).encryptor()
    return aes.update(pad(plaintext, BLOCKLEN)) + aes.finalize()

# decrypt and unpad ciphertext
def AESdecrypt(ciphertext, key, iv):
    # pick an instance of AES in CBC mode
    aes = Cipher(algorithms.AES(k), modes.CBC(iv)).decryptor()
    return unpad(aes.update(ciphertext) + aes.finalize(), BLOCKLEN)

# pretty-print encrypted blocks of data
def blocks(data):
    split = [bytes.hex(data[i:i + BLOCKLEN]) for i in range(0, len(data), BLOCKLEN)]
    return ' '.join(split)

In [5]:
from os import urandom

# pick a key
k = urandom(16)
print("k = %s" % bytes.hex(k))

# pick an IV
iv = urandom(16)
print("iv = %s" % bytes.hex(iv))

msg = b"a secret message"
ct = AESencrypt(msg, k, iv)
print("enc(%s) = %s" % (blocks(msg), blocks(ct)))
pt = AESdecrypt(ct, k, iv)
print("dec(%s) = %s" % (blocks(ct), blocks(pt)))

k = 564db74c4a234ad4cf412c610e131a0c
iv = 8780b008fe8a439ba4db3d20747f1738
enc(6120736563726574206d657373616765) = d38b245389598e95f40edb713fa065e3 03ca3b5948b3f0444576f7fb961e7d6a
dec(d38b245389598e95f40edb713fa065e3 03ca3b5948b3f0444576f7fb961e7d6a) = 6120736563726574206d657373616765


In [6]:
def prepare_cookie(bs):
    prefix = b'comment1=cooking-rocks;userdata='
    suffix = b';comment2=%20ain%27t%20give%20good%20broth'
    quoted_bs = quote_semicolons_equals(bs)
    return prefix + quoted_bs + suffix

def quote_semicolons_equals(bs):
    state = bs.decode()
    state = '";"'.join(state.split(';'))
    state = '"="'.join(state.split('='))
    return state.encode()

def decrypt_oracle(bs, k, iv):
    return is_admin(AESdecrypt(bs, k, iv))

def is_admin(bs):
    return b';admin=true;' in bs

In [7]:
pt = prepare_cookie(b"blah")
ct = AESencrypt(pt, key, iv)
# flip the least significant bit in the first byte of the ciphertext
modified_ct = bytes([ct[0] ^ 0x01]) + ct[1:]
# see the difference by XORing both ciphertexts
ct_xor_modifiedct = bytes([x ^ y for x, y in zip(ct, modified_ct)])
print("ct XOR modified_ct = %s" % blocks(ct_xor_modifiedct))

ct XOR modified_ct = 01000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000


In [8]:
# decrypt the new ciphertext
new_pt = AESdecrypt(modified_ct, k, iv)
print("before: dec(%s) = %s" % (blocks(ct), blocks(pt)))
print("after : dec(%s) = %s" % (blocks(modified_ct), blocks(new_pt)))
# see the difference in the resulting plaintext
pt_xor_newpt = bytes([x ^ y for x, y in zip(pt, new_pt)])
print("pt XOR new_pt = %s" % blocks(pt_xor_newpt))

before: dec(ff11b0ee705660c41f57514bdb4ab29d 984fdbce5092bd3135e9cdbfaca648b2 22256f2cdead92a6843c884334011792 7477b0d3bbcb1e376037c8447d2de980 733e7f9aa54f477f9e9a5c3fd50c8c2f) = 636f6d6d656e74313d636f6f6b696e67 2d726f636b733b75736572646174613d 626c61683b636f6d6d656e74323d2532 3061696e253237742532306769766525 3230676f6f6425323062726f7468
after : dec(fe11b0ee705660c41f57514bdb4ab29d 984fdbce5092bd3135e9cdbfaca648b2 22256f2cdead92a6843c884334011792 7477b0d3bbcb1e376037c8447d2de980 733e7f9aa54f477f9e9a5c3fd50c8c2f) = d301ce10e4fc7ebb4fafe93dad5b84c4 2c726f636b733b75736572646174613d 626c61683b636f6d6d656e74323d2532 3061696e253237742532306769766525 3230676f6f6425323062726f7468
pt XOR new_pt = b06ea37d81920a8a72cc8652c632eaa3 01000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 0000000000000000000000000000


In [9]:
print("\"%s\" becomes \"%s\"" % (pt[16:32].decode(), new_pt[16:32].decode()))

"-rocks;userdata=" becomes ",rocks;userdata="


In [10]:
msg = b"AadminAtrueA"
goal= b";admin=true;"
desired_diff = [x ^ y for x, y in zip(msg, goal)]
print("difference to inject into the user data: %s" % blocks(bytes(desired_diff)))

pt = prepare_cookie(msg)
ct = AESencrypt(pt, key, iv)
len_prefix = len(b'comment1=cooking-rocks;userdata=')
len_suffix = len(b';comment2=%20ain%27t%20give%20good%20broth')
assert(len_prefix == 32) #important, as the offset of userdata in next block will be 0

#      Block 1     #      Block 2     #      Block 3     #      Block 4     #      Block 5     #      Block 6     #
# comment1=cooking # -rocks;userdata= # AadminAtrueA;com # ment2=%20ain%27t # %20give%20good%2 # 0broth.......... #  plaintext
# 0000000000000000 # .00000.0000.0000 # 0000000000000000 # 0000000000000000 # 0000000000000000 # 0000000000000000 #  difference pattern injected
# 0000000000000000 # ................ # .00000.0000.0000 # 0000000000000000 # 0000000000000000 # 0000000000000000 #  affected positions

# XOR the desired diff one block before the user data (we know it is at the beginning of the previous block)
malicious_ct = ct[:16] \
               + bytes([x ^ y for x, y in zip(ct[16:16+len(desired_diff)], desired_diff)]) \
               + ct[16+len(desired_diff):]

ct_diff = bytes([x ^ y for x, y in zip(ct, malicious_ct)])
print("ct XOR malicious_ct = %s" % blocks(ct_diff))

# let the web server decrypt our cookie
malicious_pt = AESdecrypt(malicious_ct, k, iv)
print("dec(malicious_ct) = %s" % malicious_pt)
print("finally, isadmin? %s" % decrypt_oracle(malicious_ct, k, iv))

difference to inject into the user data: 7a00000000007c000000007a
ct XOR malicious_ct = 00000000000000000000000000000000 7a00000000007c000000007a00000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000
dec(malicious_ct) = b'comment1=cookingC6\x8b\x84\xe8;\x02\x19\xbd\x1e*\xb6\xec\xe3\xb1U;admin=true;;comment2=%20ain%27t%20give%20good%20broth'
finally, isadmin? True
